
# Energy Demand Forecasting Across Domains

## A Comparative Study of Local, Cross-Domain, and Global Models

## **1. Introduction**

Accurate energy demand forecasting is essential for power system planning, grid stability, and energy market operations. Modern forecasting approaches increasingly rely on machine learning models that exploit historical demand patterns, temporal features, and exogenous variables such as weather. However, a critical and often overlooked question is whether models trained in one domain can generalize effectively to another.

This project investigates domain dependence in energy demand forecasting by studying three progressively complex scenarios:

1. Forecasting city-level electricity demand (Delhi)

2. Applying the same trained model to residential electricity consumption (Pecan Street)

3. Training a single global model across both domains

The goal is not only to achieve high predictive accuracy, but to understand when, why, and how forecasting models fail—and how those failures can be addressed.


## **2. Datasets**

Two datasets were used :

1. Delhi Electricity Demand Dataset :
* Resolution - 5min (aggregated to hourly)
* Target - City-level Demand
* Features - Historical Demand, Temporal Features (hour, weekday, month), Weather variables (temperature, wind, pressure).
* Characteristics - Smooth temporal patterns, high daily seasonality, high aggregation.

2. Pecan Street Residential Dataset :
* Resolution - 1-minute (aggregated to hourly).
* Target - Household level consumption.
* Features - Historical consumption only.
* Characteristics - Varies highly, behaviour-driven, spiky and irregular demands.

## **3. Data Processing and Feature Engineering**

For both datasets, the following steps were applied:
* Conversion to hourly resolution
* Handling of missing timestamps via backward/forward fill.
* Creation of temporal features :
* Hour, weekday, month, weekend indicator, cyclical encodings using sine and cosine transformations
* Creation of autoregressive features :
* Lagged demand (1, 24, 168 hours), Rolling means (3-hour and 24-hour windows).

For residential data, weather features were not available and were set to constant values when required for feature compatibility.


## **4. Baseline Models**

1. Naive Forecast - Uses the last observed value as the forecast.
2. Seaonal Naive Forecast - Uses the demand from the same hour on the previous day.
3. Prophet - Uses the demand from the same hour on the previous day.

Results :
| Model          |   MAE |  RMSE |
| -------------- | ----: | ----: |
| Naive          | ~2723 | ~3054 |
| Seasonal Naive |  ~560 |  ~876 |
| Prophet        | ~2528 | ~2870 |


The strong performance of the seasonal naive model confirms dominant daily seasonality in city-level demand.


## **5. ML model (Model - A)**

A LightBGM model regression model was trained on Delhi dataset (model_A_lgbm.pkl)

* Tree-based gradient boosting.
* Nonlinear modeling of lagged demand and weather interactions.
* Robust to features scaling and noise.

Performance :
| Model            |        MAE |       RMSE |
| ---------------- | ---------: | ---------: |
| LightGBM (tuned) | **127.53** | **274.60** |


Short-term lags (lag_1, rolling means), daily cyclic features (hour_sin, hour_cos) were the most important.


## **6. Deep Learning and Ensemble**

An LSTM-based model was implemented to compare deep learning with tree-based methods. After proper scaling and some corrections, the LSTM underperformed LightBGM, highlighting that explicit feature engineering with tree models is more effective for structured energy time series.


## 7. Cross-Dataset Testing (Model A on Residential Data)

Model-A was applied without retraining to residential data.

Performance :
| Metric | Value |
| ------ | ----: |
| MAE    | ~1541 |
| RMSE   | ~1548 |

Predictions were mis-scaled and failed to track residential consumption patterns.

Interpretation:
* City-level demand patterns do not transfer to household-level consumption.
* Scale, variability, and drivers of demand differ fundamentally.



## **8. Retraining on Residential Data**

Scale, variability, and drivers of demand differ fundamentally.

* Correct demand scale.
* Accurate daily pattern tracking.
* Dramatically reduced error.

Confirmed that model capacity was sufficient, and the earlier failure was due to domain mismatch rather than algorithmic weakness.



## 9. Global Model Across Domains (Model-C)

A global LightGBM model was trained on a merged dataset containing both Delhi and residential data, with an added region_id feature to distinguish domains.

Result :
| Evaluation         |     MAE |    RMSE |
| ------------------ | ------: | ------: |
| Global (overall)   |  722.30 | 1011.33 |
| Delhi subset       |   96.70 |  189.24 |
| Residential subset | 1490.25 | 1494.76 |

Interpretation :
* The global model retained strong performance on Delhi data.
* Residential performance degraded significantly.
* The model prioritized the dominant, low-variance Delhi regime.

Highlights the trade-off between generalization and local optimality in multi-domain forecasting.


## **10. Key Findings**

1. Energy demand forecasting is strongly domain-dependent.
2. High local accuracy does not imply cross-domain generalization.
3. Retraining is essential when applying models across demand regimes.
4. Global models trade precision for robustness.
5. Tree-based models outperform deep learning for structured energy data.


## **11. Conclusion**

This project demonstrates that energy demand forecasting models must be adapted to their target domain. While gradient-boosted trees achieve excellent performance when trained locally, they fail to generalize across fundamentally different consumption regimes. Retraining restores performance, and global models introduce robustness at the cost of local accuracy.